In [13]:
import pandas as pd
import shapely
#import pyproj
#from pyproj import CRS
from shapely.geometry import Point, Polygon
import geopandas
import matplotlib
#from planar import BoundingBox
#from geopy import distance 
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.interpolate import griddata
import numpy as np
#import planar
import cartopy.geodesic as GD

import yaml
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection

#lettura dal file di configurazione

with open('../conf.yaml') as f:
    conf = yaml.load(f, Loader=yaml.FullLoader)
Dr = conf["uav_radius"]
Cr = conf["tau"]


## funzione per calcolare la distanza geodetica


In [2]:
#funzione per calcolare la distanza geodetica
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    #total_meters = METERS * c
    r = 6371 #radiu * 1000 to return meters
    return c * r *1000

## funzione per la divisione di punti in insieme prioritario e secondario

In [3]:
#inserisco in prio i punti che si trovano a distance < Dr 
# e li elimino da sec, invece in sec lascio i punti che si trovano ad una distanza compresa 
# tra Dr e 2*Dr
#viene richiamata dopo la funziona aggiornaDistanza in modo tale che le distante dal punto rilevante siano aggiornate
def aggiornaSecPrio(sec,prio):
    for row in sec.itertuples():
        k = row.Index
        if row.distance >= Dr*2 :
            sec.drop(k,inplace = True)
        elif row.distance <= Dr :
            #prio = geopandas.GeoDataFrame({'distance':[sec.at[k,'distance']],'geometry':[sec.at[k,'geometry']]},index=[k]).append(prio)
            prio = geopandas.GeoDataFrame({'distance':[sec.at[k,'distance']],'geometry':[sec.at[k,'geometry']], 'id_location':[sec.at[k,'id_location']]},index=[k]).append(prio)
            sec.drop(k,inplace = True)
    #import pdb; pdb.set_trace()       
    return sec,prio

## funzioni per il controllo del centro C

In [4]:
#controllo che il centro sia valido altrimenti genero un nuovo centro
def checkCenter(C,df,box,areeProibite) : 
    if invalidCenter(C,areeProibite):
        #il centro è invalido quindi provo a generare un nuovo centro
        C = generateNewCenter(df,box,areeProibite)
    return C

In [5]:
#controllo che il centro calcolato non ricade in un area proibita
def invalidCenter(C,areeProibite):
    c = Point(C[0],C[1])
    for index,value in areeProibite.items():
        if value.intersects(c):
            return True
    return False

In [6]:
#funzione che genera un nuovo centro
def generateNewCenter(df,box,areeProibite) :
    #per ogni punto in df creo un cerchio di raggio Dr, faccio l'intersezione di questi cerchi e il box
    #l'intersezione sarà uguale a tutti i possibili punti che posso scegliere come centro per coprire
    #tutti i punti in df, devo però eliminare le zone non ammesse!
    intersect_circle = box
    for row in df.itertuples():
        lon = row.geometry.x
        lat = row.geometry.y
        circle_points = GD.Geodesic().circle( lon= lon, lat=lat, radius = Dr)
        circle_poly = Polygon(circle_points)
        intersect_circle = intersect_circle.intersection(circle_poly.buffer(0))
    
    #faccio la differenza dell'intersezione calcolata prima con le zone non ammesse 
    #per trovare un'area valida all'interno della quale scegliere il nuovo centro
    for index,value in areeProibite.items():
        if intersect_circle.is_empty:
            break
        else :
            intersect_circle=intersect_circle.difference(value.buffer(0))
    #intersezione vuota -> non è stato possibile generare un nuovo centro 
    #altrimenti prendo un punto a caso in intersect_circle
    if intersect_circle.is_empty:
        return (181,91)
    else :
        # estraggo un punto ammissibile dall'area risultante
        (x,y)=intersect_circle.representative_point().coords[0]
        return (x,y)

## funzione per l'aggiornamento delle distance in df dal punto mylocation

In [7]:
#vado ad aggiornare la colonna 'distance' in df per avere le distanze tra ogni punto in df e mylocation
def aggiornaDistanza (mylocation,df):
    for row2 in df.itertuples():
            index2 = row2.Index
            location = df.at[index2,'geometry'].coords[0]
            df.at[index2,'distance'] = haversine(mylocation[1],mylocation[0],location[1],location[0])

## funzione di ottimizzazione dell'algoritmo

In [8]:
#conto quante sono le locazioni a distanza <= 2*Dr da mylocation ovvero tutte le locazioni che
#teoricamente risultano raggiungibili dal drone sapendo che si DEVE raggiungere mylocation
def insertReachableLocations (df,mylocation,myindex):
    count=0
    for row in df.itertuples():
        index = row.Index
        location=df.at[index,'geometry'].coords[0]
        d = haversine(mylocation[1],mylocation[0],location[1],location[0])
        if d <= Dr*2 :
            count=count+1
    df.at[myindex,'reachableLocations'] = count        

In [9]:
def newDfCoverage (dfInitial, dfPointCovered) : 
    dfNew = dfPointCovered.copy()
    dfNew['probability']=1
    dfNew=dfNew.drop(['distance'],axis=1)
    dfNew = dfNew.append(dfInitial)
    dfNew = dfNew.drop_duplicates(subset=['geometry'])
    return dfNew

# funzione di copertura locale 
## preso un punto restituisce la posizione ottimale del drone quando si deve coprire quel punto

In [10]:
import numpy as np
import miniball

def LocalCover(sec,myindex,box,areeProibite):
    C = sec.at[myindex,'geometry'].coords[0]
    prio = geopandas.GeoDataFrame ({'distance':[0],'geometry':[sec.at[myindex,'geometry']]}, index =[myindex])
    sec=sec.drop(myindex)
    sec = sec.drop(['reachableLocations'],axis = 'columns')
    C_checked = checkCenter (C ,prio,box,areeProibite)
    if C_checked == (181,91):
        #il punto in esame non può essere coperto in alcun caso perchè tutta la zona attorno a lui
        # è zona non ammessa
        return [(181,91),geopandas.GeoDataFrame(),0] 

    mylocation = C_checked       
    aggiornaDistanza(mylocation,sec)
    aggiornaDistanza(mylocation, prio)
    sec,prio=aggiornaSecPrio(sec,prio)
    R=prio['distance'].max()

    while sec.empty == False :
        #ordino sec in base alla colonna distance, prendo il primo elemento (quello con distanza più bassa)
        sec=sec.sort_values(by=['distance'])
        firstPosition = sec.head(1)
        #creo un nuovo dataframe con l'insieme prioritario e il primo elemento di sec
        app = geopandas.GeoDataFrame(firstPosition).append(prio)
        #elimino i duplicati altrimenti miniball genera eccezione
        app2 = app.drop_duplicates(subset=['geometry'])
        #trasformo il datafram in matrice per passarlo a miniball
        mx=app2['geometry'].x.to_numpy()
        my=app2['geometry'].y.to_numpy()
        m = np.array([mx,my]).T
        #calcolo il centro risolvendo il problema 1-center
        C, r2 = miniball.get_bounding_ball(m)
        C = tuple(C)
        #calcolo la distanza di ciascun punto di app dal centro per stabilire il raggio
        # e prepararmi a una nuova iterazione
        aggiornaDistanza(C, app)
        r = app['distance'].max()
        #r<=Dr => posso posizionare il drone per coprire tutti i punti in app
        #r>Dr => il drone non riesce a coprire tutti i punti in app quindi devo fermarmi
        if r<=Dr :
            #controllo che il centro calcolato sia valido
            #checkCenter restituisce sempre un centro C_checked che:
            # C_checked = C se C era un centro valido
            # C_checked = (181,91) = locazione inesistente se ogni centro che copre i punti in app ricade
            #in una zona non ammessa
            #altrimenti C_checked sarà uguale a un nuovo centro valido
            C_checked = checkCenter (C ,app ,box,areeProibite)
            if C_checked == (181,91):
                return [mylocation, prio ,R]
            elif C_checked!=C :
                #se è diverso da C devo riaggiornare le distanze in app
                aggiornaDistanza(C_checked,app)
            #mi preparo a una nuova iterazione: aggiorno sec, prio e le loro colonne 'distance', aggiorno 
            #la posizione e il raggio
            mylocation = C_checked
            prio = app
            sec.drop(firstPosition.index,inplace = True)
            aggiornaDistanza(mylocation,sec)
            sec,prio = aggiornaSecPrio (sec,prio)
            R = prio['distance'].max()
        else :
            return [mylocation, prio, R]
    return [mylocation,prio,R]

# algoritmo di posizionamento
## richiama la funzione LocalCover finchè non è possibile determinare la posizione ottima

In [11]:
def stationPositioning(df,box,areeProibite):
    df.reset_index(drop=True, inplace = True)
    nrows = df.index.size
    dfLocations = df.drop(['probability'], axis = 1)
    dfLocations["distance"] = ""
    numberLocations = dfLocations.shape[0]
    dfLocations["reachableLocations"] = ""
    #import pdb; pdb.set_trace()
    #ottimizzazione: conteggio i punti a distanza minore di 2*Dr da un determinato punto
    for row in dfLocations.itertuples():
        insertReachableLocations(dfLocations,row.geometry.coords[0],row.Index)
    dfLocations=dfLocations.sort_values(by=['reachableLocations'],ascending=False)
    #import pdb; pdb.set_trace()                    
    #algoritmo station-positioning
    nTott=0
    Cott = (181,91)
    #Cott = generateNewCenter(geopandas.GeoDataFrame(),box)
    Tott = None
    Rott=0
    numberIterations=0
    for row in dfLocations.itertuples():
        numberIterations = numberIterations+1
        if nTott >= row.reachableLocations :
            #i punti che potrei coprire con le prossime iterazioni non migliorerebbero il risultato
            break ;
        #richiamo LocalCover per cercare un centro C che copre il punto corrente e il maggior numero
        #di punti T
        [C,T,r] = LocalCover(dfLocations,row.Index,box,areeProibite)
        nT = T.shape[0]
        if nT == row.reachableLocations :
            #i punti teoricamente raggiungibili equivalgono a quelli raggiunti
            Cott=C
            Tott = T
            nTott = nT
            Rott=r
            break
        elif nT>nTott :
            #la soluzione appena trovata è migliore della soluzione "ottima" precedente
            Cott=C
            Tott = T
            nTott = nT
            Rott=r
        elif nT == nTott and r<Rott :
            #nel caso i punti coperti sia gli stessi scelgo la soluzione che ha raggio di copertura più basso
            Cott=C
            Tott = T
            nTott = nT
            Rott=r
            
    #output = "LA POSIZIONE OTTIMALE DEL DRONE E' " + str(Cott)+ " VENGONO INCLUSI " + str(nTott) + " PUNTI\n" +"Sono state fatte "+ str(numberIterations) + " iterazioni su " + str(numberLocations)+"\n"+str(Tott) +"\nraggio minimo = " + str(Rott) + " Dr = " + str(Dr)
    #print(output)
    return [Cott,Rott,nTott,Tott,numberIterations]

# Algoritmo di posizionamento di K stazioni
## richiama K volte l'algoritmo Station-Positioning

In [ ]:
#K-STATIONPOSITIONING
def KStationPositioning (df,box,K,areeProibite):
    df = df[(df.probability<=Cr)]
    print(Cr)
    print("Kstation: after filter :",len(df))
    #print(df)
    #inizializzazione dei valore da restituire
    maxIterations = 0
    numberPoints = df.shape[0]
    numberIterations = 0
    i = 0
    list_centers = []
    sum_nTott = 0
    sum_Tott = geopandas.GeoDataFrame()
    list_radius = []
    #se le stazioni non sono state tutte posizionate e ci sono ancora punti da coprire richiamo
    #l'algoritmo StationPositioning per ottenere:
    # C = posizione del drone
    # R = raggio minimo per coprire tutti i punti in T
    # T = tutti i punti coperti avendo centro C
    # nT = numero di punti coperti
    # nIter = numero di iterazioni di LocalCover 
    while i<K and df.empty==False:
        [C,R,nT,T,nIter]=stationPositioning(df,box,areeProibite)
        maxIterations = maxIterations + df.shape[0]
        numberIterations = numberIterations + nIter
        if C == (181,91):
            #non è stato possibile posizionare questa stazione quindi non sarà possibile posizionarne 
            #altre perchè i punti non ancora coperti non possono essere coperti a causa delle zone non ammesse
            break
        #aggiornamento valori da restituire
        list_centers.append(C)
        list_radius.append(R)
        sum_Tott = sum_Tott.append(T)
        print("Tot k",i,len(T))
        sum_nTott = sum_nTott+nT
        #creo un cerchio di centro C e raggio Dr
        circle_points = GD.Geodesic().circle( lon=C[0] , lat=C[1] , radius = Dr)
        circle_poly = Polygon(circle_points)
        dfCircle = geopandas.GeoDataFrame({'geometry':[circle_poly]},crs="EPSG:4326")
        #faccio la differenza tra il df contente i punti e il cerchio in modo da eliminare i punti già
        # coperti
        df = geopandas.overlay(df,dfCircle,how="difference")
        i=i+1
      
    return [list_centers,list_radius,sum_nTott,sum_Tott,numberIterations,maxIterations,numberPoints,K]
